# <h1><font color="#f37626">[Lifecycle]</font> autolog-keras 예제 코드
----
`autolog()`는 모델링 과정 중 실험 이력으로 저장하고자 하는 모델을 자동으로 accuinsight+ modeler 콘솔 화면에 기록해주는 메소드 입니다.

- data: IMDB 영화 리뷰 
- keras 사용  


`autolog()`는 다음의 과정을 통해 실험 이력을 기록합니다.
1. 임의의 변수에 `accuinsight()` 할당
2. model.fit() 이전에 `autolog()` 선언
3. model.fit()을 사용하여 모델 학습 진행     

----

### 1. Accuinsight()
- accuinsight() 객체를 생성합니다.

In [1]:
from Accuinsight.Lifecycle.keras import accuinsight

accu = accuinsight()

Using TensorFlow backend.


#### [option 1] DataCatalog에서 데이터 다운로드
- DataCatalog에서 관리하는 데이터를 받아올 수 있습니다.  

    1. Data catalog 서비스 화면에서 다운로드하려는 데이터의 경로를 __copy location__ 버튼을 클릭하여 복사합니다.
    2. 복사한 경로를 `get_file()`의 argument로 입력합니다.
    3. info의 출력값인 데이터가 저장된 경로를 사용하여 데이터를 불러올 수 있습니다. 

In [ ]:
dataCatalog_endpoint = 'location'

accu.get_file(dataCatalog_endpoint)

#### [option 2] Slack 메시지 푸시 
1. slack api에서 발급받은 token과 메시지를 받아보고자 하는 channel의 id를 입력합니다.  
`set_slack(token, channel_id)`

2. slack으로 message를 푸시하는 방법은 다음과 같습니다.
    - 모델 학습 완료시   
    `send_message('your_message')`

    - 학습에 사용되는 metric이 일정 thresholds를 넘은 경우  
    `send_message(thresholds = 0.5)`
    
> 두 가지 방법을 동시에 사용할 수 없습니다.

In [2]:
token = 'your_token'
cid = 'your_channel_id'

accu.set_slack(token = token, channel_id = cid)

In [3]:
# accu.send_message(message = '[DL-classification-keras] 모델 학습 완료')

accu.send_message(thresholds = 0.1)  ### 예제에서는 thresholds를 넘을 경우에만 메시지를 푸시합니다.

### 2. Load data
- __target__  
(긍정: 1 / 부정: 0)

In [4]:
from keras.datasets import imdb
import numpy as np

# train & test data
(train_data, train_label), (test_data, test_label) = imdb.load_data(num_words=10000)

word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in train_data[0]])

X_train = np.zeros((len(train_data), 10000))
for i, sequence in enumerate(train_data):
    X_train[i, sequence] = 1. 
    
X_test = np.zeros((len(train_data), 10000))
for i, sequence in enumerate(test_data):
    X_test[i, sequence] = 1. 


y_train = np.asarray(train_label).astype('float32')
y_test = np.asarray(test_label).astype('float32')


# validation data 
x_val = X_train[:10000]
partial_x_train = X_train[10000:]

y_val = y_train[:10000]
partial_y_train = y_train[10000:]

### 3. Model compile

In [5]:
from keras.models import Sequential, Model
from keras.layers import Dense
from keras import optimizers
from keras import metrics

def build_basic_model():
    base_model = Sequential()
    base_model.add(Dense(16, activation='relu', input_shape=(10000,)))
    base_model.add(Dense(16, activation='relu'))
    base_model.add(Dense(1, activation='sigmoid'))
    
    return base_model

opt = optimizers.RMSprop(lr=0.0001)
model = build_basic_model()

## model compile
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[metrics.BinaryCrossentropy()])

### 4. autolog() 
- `autolog()`에 현재 학습할 모델에 관한 간단한 tag를 입력할 수 있습니다.
- `autolog()`는 반드시 모델 학습(model.fit()) __이전에__ 호출이 되어야 합니다.

In [6]:
accu.autolog('Keras-binary-classification')

### 5. model fitting

In [7]:
model.fit(partial_x_train, partial_y_train, epochs=50, batch_size=512, validation_data=(x_val, y_val))

Train on 15000 samples, validate on 10000 samples
Epoch 1/50
15000/15000 [==============================] - 2s 126us/step - loss: 0.6748 - binary_crossentropy: 0.6748 - val_loss: 0.6529 - val_binary_crossentropy: 0.6529

Epoch 00001: val_binary_crossentropy improved from inf to 0.65293, storing weights.

Epoch 2/50
15000/15000 [==============================] - 1s 84us/step - loss: 0.6224 - binary_crossentropy: 0.6224 - val_loss: 0.6033 - val_binary_crossentropy: 0.6033

Epoch 00002: val_binary_crossentropy improved from 0.65293 to 0.60328, storing weights.

Epoch 3/50
15000/15000 [==============================] - 1s 84us/step - loss: 0.5666 - binary_crossentropy: 0.5666 - val_loss: 0.5529 - val_binary_crossentropy: 0.5529

Epoch 00003: val_binary_crossentropy improved from 0.60328 to 0.55292, storing weights.

Epoch 4/50
15000/15000 [==============================] - 1s 84us/step - loss: 0.5131 - binary_crossentropy: 0.5131 - val_loss: 0.5060 - val_binary_crossentropy: 0.5060

Epoch 

__모델 학습이 완료되면 `autolog()`는 자동으로 해제됩니다.__  
따라서 모델 학습 이력을 추가하고자 할 경우, 다시 한 번 `autolog()` 호출 후 모델 훈련을 진행해야 합니다.

### 6. 저장된 모델 불러오기
- `autolog()`를 사용하여 모델의 학습 이력을 Lifecycle에 기록할 경우, 자동으로 훈련 도중 가장 좋은 metric을 기록한 epoch에서의 모델 가중치가 저장됩니다.
- 따라서 저장된 모델을 불러와 공동 작업자들과 모델을 공유하거나, 모델 재학습을 수행할 수 있습니다.

    1. Accuinsight+ workspace list 혹은 해당 모델의 상세화면으로 접속하여 _Experiment_ 중 불러오고자 하는 모델의 __Run name__을 복사합니다.
    2. ___utils___에서 `load_model()` 함수를 호출하여 모델을 불러올 수 있습니다.

In [8]:
from Accuinsight.Lifecycle.utils import load_model

saved_model = load_model('keras-A9E821A55D5F4CA9A78D86827C62AA62_13')

In [9]:
saved_model.get_weights()

[array([[-0.02157974,  0.01793922, -0.00658817, ...,  0.01585907,
          0.00722466,  0.02340497],
        [ 0.01130816,  0.02808222, -0.00071141, ..., -0.01401629,
         -0.01594194,  0.00995289],
        [-0.01384417,  0.01335631,  0.01874032, ...,  0.00270803,
          0.00705006,  0.03091275],
        ...,
        [-0.02350396, -0.0181123 , -0.0004642 , ..., -0.01544702,
          0.00950608,  0.00926542],
        [ 0.01578502, -0.01753234,  0.00388583, ..., -0.0154772 ,
          0.01169709,  0.00266412],
        [ 0.01752615, -0.0051559 ,  0.01353151, ...,  0.01527549,
          0.00647289,  0.02042266]], dtype=float32),
 array([ 0.00769699,  0.00435105,  0.00520558,  0.0025067 ,  0.00225739,
        -0.00069461,  0.00018096,  0.00558622,  0.00441365,  0.00477135,
         0.00354772,  0.01000588,  0.00634174,  0.00376313,  0.00469057,
         0.00822093], dtype=float32),
 array([[-1.66987658e-01, -3.34941030e-01,  2.75082707e-01,
          3.62039864e-01,  3.77821684e-01